In [3]:
import os
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler
from sacred import Experiment
from argparse import Namespace
from torchvision import transforms

from pytorchtools import EarlyStopping

from analysis import plot_loss
from load_data.eye_dataset import EyeDataset, EyeDatasetOverfitCorners, EyeDatasetOverfitCenter
import logging
import numpy as np
from models.fc import FC
from models.unet import UNET
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

logger = logging.getLogger(__name__)

ex = Experiment('EyeSegnmentation', interactive=True)
ex.logger = logger

%matplotlibe inline

@ex.config
def cfg():
    data_path = 'data/drive/training'
    data_path_training = 'data/drive/training'
    data_path_validation = 'data/drive/validation'
    data_path_test = 'data/drive/test'
    num_epochs = 10000
    batch_size = 1
    plot_loss = True
    checkpoint_path = 'checkpoints/v1'
    is_save_model = False
    #model_load_path = None
    model_load_path = 'checkpoints/v1/20190609-150526'
    display_images = True
    
    
@ex.main
def main(_run):
    args = Namespace(**_run.config)
    logger.info(args)

    # ------ Michals modification: split train and validation in advance ------ #
    # train and validation images should be placed in args.data_path_training and args.data_path_validation
    # last 4 images (#37-40) are used as validation
    loader_train = EyeDatasetOverfitCenter(args.data_path_training, augment=True, normalization=True)
    loader_val = EyeDatasetOverfitCenter(args.data_path_validation, augment=True, normalization=True)
    loader_test = EyeDatasetOverfitCenter(args.data_path_test, augment=True, normalization=True)

    # loader = EyeDataset(args.data_path, augment=True)
    ## training_data = DataLoader(loader, shuffle=True, batch_size=1, sampler=train_sampler)
    # training_data, test_data = split_dataset_to_train_and_test(loader, args.batch_size)

    training_data = DataLoader(loader_train, batch_size=args.batch_size)
    validatoin_data = DataLoader(loader_val, batch_size=args.batch_size)
    test_data = DataLoader(loader_test, batch_size=args.batch_size)
    # ------------------------------------------------------------------------- #

    model = choose_model(args, training_data)
    # TEST
    print("evaluate on training data")
    evaluate_results(args, model, training_data)
    print("evaluate on validation data")
    evaluate_results(args, model, validatoin_data)
    # print("evaluate on test data")
    # evaluate_results(args, model, test_data)


UsageError: Line magic function `%matplotlibe` not found.
